In [45]:
''''
For debugging purposes, copy and modify functions here (fixing bugs)
'''

"'\nFor debugging purposes, copy and modify functions here (fixing bugs)\n"

In [46]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt


import warnings
from warnings import filterwarnings


from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_validate, RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.exceptions import DataConversionWarning


In [47]:
# Load the dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from DataPreparation.Ingestion.Ingestion import read_data, visualize_data
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import sys

from DataPreparation.Preprocessing.Preprocessing import preprocess_data

df = read_data(path='./DataFiles/house-prices-advanced-regression-techniques')

x_train_p, y_train, x_val_p, y_val = preprocess_data(df)


columns having only 2 unique values: 3
['Street', 'Utilities', 'CentralAir']
columns having only <=10 unique values: 37
['MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
columns having only <=10 unique values: 3
['Neighborhood', 'Exterior1st', 'Exterior2nd']


In [48]:
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe


binary_cols = [col for col in df.columns if df[col].dtype not in [
    "int64", "float64"] and df[col].nunique() == 2]
print(binary_cols)
for col in binary_cols:
    label_encoder(df, col)


[]


In [49]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=True):
    dataframe = pd.get_dummies(
        dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe


ohe_cols = [col for col in df.columns if 10 >= df[col].nunique() > 2]
print(ohe_cols)

df = one_hot_encoder(df, ohe_cols)

df.head()

['MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'YrSold', 'SaleType', 'SaleCondition']


,MSSubClass,LotFrontage,LotArea,Street,Utilities,Neighborhood,YearBuilt,YearRemodAdd,Exterior1st,Exterior2nd,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,1,0,CollgCr,2003,2003,VinylSd,VinylSd,...,False,False,False,False,True,False,False,False,True,False
1,20,80.0,9600,1,0,Veenker,1976,1976,MetalSd,MetalSd,...,False,False,False,False,True,False,False,False,True,False
2,60,68.0,11250,1,0,CollgCr,2001,2002,VinylSd,VinylSd,...,False,False,False,False,True,False,False,False,True,False
3,70,60.0,9550,1,0,Crawfor,1915,1970,Wd Sdng,Wd Shng,...,False,False,False,False,True,False,False,False,False,False
4,60,84.0,14260,1,0,NoRidge,2000,2000,VinylSd,VinylSd,...,False,False,False,False,True,False,False,False,True,False


In [50]:
useless_cols = [col for col in df.columns if df[col].nunique() == 2 and
                (df[col].value_counts() / len(df) < 0.01).any(axis=None)]

# useless_cols = [col for col in useless_cols if col not in ['Id', 'SalePrice']]

df.drop(useless_cols, axis=1, inplace=True)

print(useless_cols)

['Street', 'Utilities', 'LotShape_IR3', 'LotConfig_FR3', 'LandSlope_Sev', 'Condition1_PosA', 'Condition1_RRAe', 'Condition1_RRNe', 'Condition1_RRNn', 'Condition2_Feedr', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'HouseStyle_1.5Unf', 'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'OverallQual_2', 'OverallCond_2', 'RoofStyle_Gambrel', 'RoofStyle_Mansard', 'RoofStyle_Shed', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl', 'ExterQual_Fa', 'ExterCond_Po', 'Foundation_Stone', 'Foundation_Wood', 'BsmtCond_Po', 'BsmtFinType2_GLQ', 'Heating_Grav', 'Heating_OthW', 'Heating_Wall', 'HeatingQC_Po', 'Electrical_FuseP', 'Electrical_Mix', 'BsmtFullBath_3', 'BsmtHalfBath_2', 'HalfBath_2', 'BedroomAbvGr_6', 'BedroomAbvGr_8', 'KitchenAbvGr_3', 'Functional_Maj2', 'Functional_Sev', 'Fireplaces_3', 'GarageType_CarPort', 'GarageCars_4', 'GarageQual_Gd', 'GarageQual_Po', 'GarageCond_Gd', 'GarageCo

In [39]:
df.drop(["Neighborhood","Exterior1st", "Exterior2nd"], axis=1, inplace=True)

In [51]:
def cross_val(model):
    pred = cross_val_score(model, x_train_p, y_train, cv=5)
    return pred.mean()


def plot_importance(model, features, num=len(x_train_p), save=False):
    feature_imp = pd.DataFrame(
        {'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')


In [44]:
# Random Forest Regressor
rf = RandomForestRegressor(random_state=17)

rf_random_params = {"max_depth": np.random.randint(5, 50, 10),
                    "max_features": [3, 5, 7, "auto", "sqrt"],
                    "min_samples_split": np.random.randint(2, 50, 20),
                    "n_estimators": [int(x) for x in np.linspace(start=200, stop=1500, num=10)]}

rf_random = RandomizedSearchCV(estimator=rf,
                               param_distributions=rf_random_params,
                               n_iter=100,
                               cv=3,
                               verbose=True,
                               random_state=42,
                               n_jobs=-1)

rf_random.fit(x_train_p, y_train)


ValueError: Found input variables with inconsistent numbers of samples: [1103, 1168]

In [ ]:
# Print columns with string values
print(df.columns[df.dtypes == "object"])
